In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import sys
sys.path.append('../models')
from vgg import VGG16_model

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


### Load the Pocovid Dataset

In [4]:
import sys
sys.path.append('../scripts')
from pocovid_dataset import PocovidDataset
from torch.utils.data import Dataset, DataLoader

In [7]:
train_pct = 0.8
batch_size = 1
num_workers = 2

dataset = PocovidDataset(root_dir='../data/image_dataset',
                                     transform=transforms.Compose([
                                       transforms.Resize((224,224)),
                                       transforms.RandomAffine(10,translate=(0.1,0.1)),
                                       transforms.ToTensor()
                                       ]))

dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=num_workers)

train_len = int(len(dataloader) * train_pct)

trainset, valset = torch.utils.data.random_split(dataset, [train_len, len(dataset) - train_len])

train_loader = torch.utils.data.DataLoader(trainset, num_workers=num_workers, shuffle=True,
                                  batch_size=batch_size)

val_loader = torch.utils.data.DataLoader(valset, num_workers=num_workers, shuffle=False,
                                batch_size=batch_size)

In [10]:
len(dataset)

3257

In [6]:
class_map = {'covid' : dataset.get_covid_class_idx(),
            'pneumonia' : dataset.get_pneu_class_idx(),
            'regular' : dataset.get_regular_class_idx()}

In [14]:
train_loader

In [15]:
for data in (train_loader):
    print(data)
    break

KeyboardInterrupt: 

### Model

In [7]:
model = VGG16_model()

### Loss function and Optimizer

In [8]:
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(params = model.parameters(), lr=1e-4)

### Training Loop

In [9]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        print(i, data)
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        print(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

print('Finished Training')

KeyboardInterrupt: 